In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# 모델 저장 경로
MODEL_PATH = "./ToneDetect_model"

# 토크나이저와 학습된 모델 불러오기
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
model = BertForSequenceClassification.from_pretrained(MODEL_PATH)
model.eval()

c:\Users\김소연\ToneDetect\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# 파일 입력 받기 (KakaoTalk_20250511_1625_29_658_유정유정)

# 파일의 형식은 아래와 같음
# --------------- 2025년 3월 19일 수요일 ---------------
# [유정유정] [오후 2:02] 언니 혹시 끝났어??
# [김소연] [오후 2:03] 아직 수업중ㅠㅠ
# [유정유정] [오후 2:03] 헉!!
# [유정유정] [오후 2:03] 먼저 가고 있을게 ㅎㅎ
# [김소연] [오후 2:03] 웅웅~
# [유정유정] [오후 2:08] 오른쪽 앞에서 두번째 안쪽으로 자리 잡아놔쓰😆❤
# [김소연] [오후 2:16] 나 방금 끝나서 가는 중!
# --------------- 2025년 3월 26일 수요일 ---------------
# [유정유정] [오후 2:16] ㅎㅎ 언니 미안... 또 앞자리얌...
# [유정유정] [오후 2:16] 저번에 앉았던 그 자리...^^

# 날짜 부분 버리기
# 이름 별로 문장 모으기

# 기존 코드와 합치기
# 최종 결과는 이름 당 각 클래스로 예측된 문장 갯수
# 예시)
# 김소연 -> {로봇 말투 (android): 5, 연장자 말투(azae):3 ...}
# 유정유정 -> {로봇 말투 (android): 2, 연장자 말투(azae):7 ...}


In [5]:
text = "안녕하세요"  # 예측하고 싶은 문장

# 입력 토크나이즈
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# 예측
with torch.no_grad():
    outputs = model(**inputs)   
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

label_map = {
    0: "로봇 말투 (android)",
    1: "연장자 말투 (azae)",
    2: "채팅체 (chat)",
    3: "초등학생 말투 (choding)",
    4: "이모티콘 포함 (emoticon)",
    5: "외향적 말투 (enfp)",
    6: "존댓말 (formal)",
    7: "극존칭 (gentle)",
    8: "할아버지 말투 (halbae)",
    9: "욕쟁이 할머니 말투 (halmae)",
    10: "반말 (informal)",
    11: "중2병 말투 (joongding)",
    12: "왕 말투 (king)",
    13: "특정 어미 부착 (naruto)",
    14: "선비 말투 (seonbi)",
    15: "소심한 말투 (sosim)",
    16: "번역기 말투 (translator)"
}

print("예측된 클래스:", predicted_class)
print("예측된 말투:", label_map[predicted_class])


예측된 클래스: 6
예측된 말투: 존댓말 (formal)


In [9]:
import torch
from collections import defaultdict
import re

# ✅ 말투 클래스 맵
label_map = {
    0: "로봇 말투 (android)",
    1: "연장자 말투 (azae)",
    2: "채팅체 (chat)",
    3: "초등학생 말투 (choding)",
    4: "이모티콘 포함 (emoticon)",
    5: "외향적 말투 (enfp)",
    6: "존댓말 (formal)",
    7: "극존칭 (gentle)",
    8: "할아버지 말투 (halbae)",
    9: "욕쟁이 할머니 말투 (halmae)",
    10: "반말 (informal)",
    11: "중2병 말투 (joongding)",
    12: "왕 말투 (king)",
    13: "특정 어미 부착 (naruto)",
    14: "선비 말투 (seonbi)",
    15: "소심한 말투 (sosim)",
    16: "번역기 말투 (translator)"
}
# 클래스 이름을 → 인덱스로 역변환 (검색용)
reverse_label_map = {v: k for k, v in label_map.items()}

# ✅ 사용자별 말 모으기
def parse_chat_file(filename):
    user_messages = defaultdict(list)
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if re.match(r"^-{5,}", line) or re.match(r"^\d{4}년 \d{1,2}월 \d{1,2}일", line):
                continue
            match = re.match(r"\[([^]]+)] \[([^]]+)] (.+)", line)
            if match:
                user, time, message = match.groups()
                user_messages[user].append(message)
    return user_messages

# ✅ 말투 예측 함수
def predict_style(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class

# ✅ 전체 분석 및 저장 (문장 포함)
def analyze_chat_styles_with_messages(filename):
    user_messages = parse_chat_file(filename)
    result_summary = {}
    result_sentences = {}

    for user, messages in user_messages.items():
        style_counts = defaultdict(int)
        style_messages = defaultdict(list)
        for message in messages:
            predicted_class = predict_style(message)
            style_name = label_map[predicted_class]
            style_counts[style_name] += 1
            style_messages[style_name].append(message)
        result_summary[user] = dict(style_counts)
        result_sentences[user] = dict(style_messages)
    
    return result_summary, result_sentences

# ✅ 조회 함수: 사용자 + 말투 클래스 → 문장 리스트 출력
def get_sentences_by_style(user, style_name, result_sentences):
    if user not in result_sentences:
        print(f"[오류] 사용자 '{user}'를 찾을 수 없습니다.")
        return
    if style_name not in result_sentences[user]:
        print(f"[알림] '{user}'는 '{style_name}' 말투로 분류된 문장이 없습니다.")
        return
    print(f"\n[{user}]의 '{style_name}' 예측 문장들:")
    for idx, sentence in enumerate(result_sentences[user][style_name], 1):
        print(f"{idx}. {sentence}")


In [10]:
# ✅ 실행 예시
filename = "KakaoTalk_20250511_1625_29_658_유정유정.txt"
result_summary, result_sentences = analyze_chat_styles_with_messages(filename)

# ✅ 전체 요약 결과 출력
print("\n[전체 사용자별 예측 결과 요약]")
for user, style_count in result_summary.items():
    print(f"\n{user} ->")
    for style, count in style_count.items():
        print(f"  {style}: {count}")


[전체 사용자별 예측 결과 요약]

김소연 ->
  존댓말 (formal): 2
  번역기 말투 (translator): 2
  외향적 말투 (enfp): 80
  중2병 말투 (joongding): 12
  소심한 말투 (sosim): 18
  선비 말투 (seonbi): 2
  할아버지 말투 (halbae): 7
  채팅체 (chat): 33
  이모티콘 포함 (emoticon): 8
  반말 (informal): 5
  초등학생 말투 (choding): 10
  욕쟁이 할머니 말투 (halmae): 3
  연장자 말투 (azae): 3
  극존칭 (gentle): 2

유정유정 ->
  존댓말 (formal): 3
  번역기 말투 (translator): 6
  초등학생 말투 (choding): 28
  외향적 말투 (enfp): 120
  연장자 말투 (azae): 8
  채팅체 (chat): 47
  할아버지 말투 (halbae): 14
  이모티콘 포함 (emoticon): 11
  중2병 말투 (joongding): 19
  극존칭 (gentle): 2
  욕쟁이 할머니 말투 (halmae): 4
  왕 말투 (king): 3
  반말 (informal): 2
  소심한 말투 (sosim): 4
  선비 말투 (seonbi): 2


In [ ]:
# ✅ 원하는 문장 보기
get_sentences_by_style("김소연", "채팅체 (chat)", result_sentences)


[김소연]의 '채팅체 (chat)' 예측 문장들:
1. 나 방금 끝나서 가는 중!
2. 아니 오늘 진도 반도 못나간것 같은데ㅋㅋㅋㅋㅋㅋㅋ
3. 알았또
4. 왜 안보이징
5. 헐 용량 크다구 카톡으로도 안보내진다...ㅠㅠ
6. 알았또
7. zip으로 다운받지 말구~
8. 아이디 알려주면 너 초대해줄겡
9. 오키~
10. 그르겡ㅋㅋㅋ
11. 나 뭐하라고 했는지 기억도 잘 안나는데..ㅋㅋㅋㅋ
12. 앗ㅋㅋㅋㅋ 나 202304198이야!
13. 유정 나 이제 집 도착ㅠ ppt 시작하려궁
14. 구현할 내용에서 카카오톡 .txt 파일 업로드하는거 설명하고 싶어서!
15. 제출 한 50분 남기고 할말은 아니긴 한데..ㅎㅎ
16. 근데 어미나 이런거 분석할 때 문장을 단어 단위로 나누고 단어들을 비지도학습으로 클러스터링 하면 어떨까 하는 생각이 들었어..ㅋㅋ
17. 나는 왜 안보이징..ㅜㅠ
18. 아니 두번째 동그라미 띄어쓰기 엄청 되어있는거 왜 그랬던거야..?ㅋㅋ
19. 조아조아~
20. ㅋㅋㅋㅋ천천히 해~
21. 괜찮겠지..??ㅋㅋㅋㅋ
22. 진짜 뭐지..ㅋㅋㅋㅋㅋ
23. ㅋㅋㅋㅋㅋㅋㅋㅋㅋ그럴리강
24. 헐랭 빨리 자..
25. 헐 구랭??ㅋㅋㅋ
26. 헐..
27. 방금 봤어ㅋㅋㅋㅋㅋ
28. 난 지각할듯..ㅎㅎ
29. 근데 저번에 교수님이 발표 딱 언제라고 얘기한적은 없으시고
30. 수인분당선역 근처에서 보면 좋을것 같은뎅..
31. 근데 태평역 근처에 같이 팀플 할만한데가 딱히 없당ㅋㅋㅋㅋ
32. 금방 가ㅋㅋㅋ
33. 아하
